<a href="https://colab.research.google.com/github/ccarpenterg/LearningTensorFlow2.0/blob/master/05_pretrained_convnets_and_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretrained Convolutional Networks

Before trying to tackle the problem of overfitting, we will explore the concept of a pre-trained convnet. So this is where the world of AI gets really exciting. We not only can use a great ML framework like TensorFlow, developed by Google, one the most advanced companies in terms of AI, but we also can download a pretrained convolutional neural network, that has been trained by a company like Google or by a research institution like Stanford.


That means that years and years of research are **available** to everybody, provided they have the technical skillls to use these pretrained convolutional neural networks.

Let's start by installing tensorflow 2.0 and checking we have the right version:



In [0]:
!pip install tensorflow==2.0.0-beta1

In [0]:
import tensorflow

print(tensorflow.__version__)

2.0.0-beta1


## VGG16

The VGG16 network was developed by the Visual Geometry Group at Oxford University, hence the name VGG. It has 16 layers, including convolutional and fullly connected layers, and a little more than 138 million parameters.

We can download the convnet structure and parameters through the Keras module by instantiating the class VGG16:



In [0]:
from tensorflow.keras.applications import VGG16

convnet = VGG16()

convnet.summary()

553467904/553467096 [==============================] - 40s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

So this is pretty much like downloading a brain, or a section a of a brain to be more precise. Now keep in mind that we have to re-train parts of a pretrained network so that we can apply the knowledge of this network to our particular problem.

Most of the time the pretrained convnet has been trained on large datatasets such as Imagenet and with very different classes, so in order to apply it to our particular problem we will have to remove the dense classifier, which are the fully connected layers that sit on top of the convolutional and polling layers.

### Feature Extraction and Fine-tuning

So why do we want to use a pre-trained convnet if its cassifier is not able to classify our images? The answer is **Feature extraction**. A convnet's architecture consists of two parts: the convolutional (and pooling) layers, and the classifier (fully connected layers).

If we use the analogy of the human brain, we can say that the visual cortex is equivalent to our convolutional layers. They extract features like horizontal lines, vertical lines, curves, edges, etc.

In the fundamental level, what the convnet is doing is extracting features like horizontal lines, vertical lines




In [0]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))

conv_base.summary()

58892288/58889256 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

### Freezing and Unfreezing Layers



### Other Pretrained Convnets



*   Xception
*   ResNet
*   Inception
*   MobileNet
*   DenseNet
*   NASNet

